## TRAINING MODEL

In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler, ModelCheckpoint, EarlyStopping
from keras.optimizers import SGD
from cnn import config
from cnn.resnet import ResNet
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse

# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")
 
# construct the argument parser and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-p", "--plot", type=str, default="plot.png", help="path to output loss/accuracy plot")
args = vars(ap.parse_args())

Using TensorFlow backend.
usage: ipykernel_launcher.py [-h] [-p PLOT]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\ANKIT SUMAN\AppData\Roaming\jupyter\runtime\kernel-798b9d12-6c96-424e-bdc9-9847a7eede38.json


SystemExit: 2

c:\users\ankit suman\appdata\local\programs\python\python36\lib\site-packages\IPython\core\interactiveshell.py:3275: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## Define the # of epochs, initial learning rate and batch size.

In [2]:
num_epochs = 100
init_lr= 1e-1
bs = 512
 
# create a function called polynomial decay which helps us decay our 
# learning rate after each epoch

def poly_decay(epoch):
    
	# initialize the maximum # of epochs, base learning rate,
	# and power of the polynomial
	maxEpochs = num_epochs
	baseLR = init_lr
	power = 1.0  # turns our polynomial decay into a linear decay
 
	# compute the new learning rate based on polynomial decay
	alpha = baseLR * (1 - (epoch / float(maxEpochs))) ** power
 
	# return the new learning rate
	return alpha

## Determine the # of image paths in training/validation/testing directories.


In [3]:
totalTrain = len(list(paths.list_images(config.train_path)))
totalVal = len(list(paths.list_images(config.val_path)))
totalTest = len(list(paths.list_images(config.test_path)))

### Initialize the training data augmentation object. randomly shifts, translats, and flips each training sample.

In [4]:

trainAug = ImageDataGenerator(
	rescale=1 / 255.0,
	rotation_range=20,
	zoom_range=0.05,
	width_shift_range=0.05,
	height_shift_range=0.05,
	shear_range=0.05,
	horizontal_flip=True,
	fill_mode="nearest")

### Initialize the validation (and testing) data augmentation object.

In [5]:
valAug = ImageDataGenerator(rescale=1 / 255.0)

In [6]:
# initialize the training generator
trainGen = trainAug.flow_from_directory(
	config.train_path,
	class_mode="categorical",
	target_size=(64, 64),
	color_mode="rgb",
	shuffle=True,
	batch_size=bs)
 
# initialize the validation generator
valGen = valAug.flow_from_directory(
	config.val_path,
	class_mode="categorical",
	target_size=(64, 64),
	color_mode="rgb",
	shuffle=False,
	batch_size=bs)
 
# initialize the testing generator
testGen = valAug.flow_from_directory(
	config.test_path,
	class_mode="categorical",
	target_size=(64, 64),
	color_mode="rgb",
	shuffle=False,
	batch_size=bs)

Found 19842 images belonging to 2 classes.
Found 2204 images belonging to 2 classes.
Found 5512 images belonging to 2 classes.


In [7]:
# initialize our ResNet model and compile it
model = ResNet.build(64, 64, 3, 2, (3, 4, 6), (64, 128, 256, 512), reg=0.0005)
opt = SGD(lr=init_lr, momentum=0.9)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

In [8]:
model.summary()

filepath = model.name + '.{epoch:02d}-{val_loss:.4f}-{val_acc:.4f}.h5'

# Define the model checkpoint callback -> this will keep on saving the model as a physical file
#----------------------------------------------------------------------------------------------
model_checkpoint = ModelCheckpoint(filepath, verbose=1, save_best_only=True)


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 64, 64, 3)    12          input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 64, 64, 64)   4800        batch_normalization_1[0][0]      
__________________________________________________________________________________________________
batch_normalization_2 (BatchNor (None, 64, 64, 64)   256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation

### Define our set of callbacks and fit the model.

In [9]:
callbacks = [LearningRateScheduler(poly_decay), model_checkpoint]

H = model.fit_generator(
	trainGen,
	steps_per_epoch=totalTrain // bs,
	validation_data=valGen,
	validation_steps=totalVal // bs,
	epochs=num_epochs,
	callbacks=callbacks)

Epoch 1/100


ResourceExhaustedError: OOM when allocating tensor with shape[512,64,32,32] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node conv2d_12/convolution}} = Conv2D[T=DT_FLOAT, _class=["loc:@training/SGD/gradients/conv2d_12/convolution_grad/Conv2DBackpropInput"], data_format="NCHW", dilations=[1, 1, 1, 1], padding="VALID", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](activation_11/Relu, conv2d_12/kernel/read)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[{{node metrics/acc/Mean_1/_1153}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_11398_metrics/acc/Mean_1", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:C

In [ ]:
# reset testing generator and use the trained model to make predictions 
print("[INFO] evaluating network...")
testGen.reset()
predIdxs = model.predict_generator(testGen,
	steps=(totalTest // bs) + 1)
 
# finds the index of the label with largest predicted probability of each
# testing image 

predIdxs = np.argmax(predIdxs, axis=1) 
 
# displays a classification report
print(classification_report(testGen.classes, predIdxs,
	target_names=testGen.class_indices.keys()))

In [ ]:
# plot the training loss and accuracy
N = num_epochs
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig(args["plot"])

In [11]:
import pandas as pd
from keras.models import load_model

from PIL import Image
from keras.preprocessing import image

In [12]:
testing = pd.read_csv('submission.csv')  

model = load_model('resnet.71-0.2467-0.9718.h5')

In [15]:

with open('submission_format.csv', 'w') as csvFile:
        csvFile.write('{},{}'.format('filename','infected'))

for i in range(len(testing.filename)):
    
    img = image.load_img('test/'+testing.filename[i], target_size=(64,64,3), color_mode='rgb')
    img = image.img_to_array(img)
    img = img/255
    
    data = np.array([img.reshape( (64, 64, 3) )])
        
    get_data = model.predict(data)
    


    print('written: '+ testing.filename[i])
    
    with open('submission_format.csv', 'a') as csvFile:
        csvFile.write('\n{},{}'.format(testing.filename[i], get_data[0][0]))

written: 21765.jpg
written: 21766.jpg
written: 21767.jpg
written: 21768.jpg
written: 21769.jpg
written: 21770.jpg
written: 21771.jpg
written: 21772.jpg
written: 21773.jpg
written: 21774.jpg
written: 21775.jpg
written: 21776.jpg
written: 21777.jpg
written: 21778.jpg
written: 21779.jpg
written: 21780.jpg
written: 21781.jpg
written: 21782.jpg
written: 21783.jpg
written: 21784.jpg
written: 21785.jpg
written: 21786.jpg
written: 21787.jpg
written: 21788.jpg
written: 21789.jpg
written: 21790.jpg
written: 21791.jpg
written: 21792.jpg
written: 21793.jpg
written: 21794.jpg
written: 21795.jpg
written: 21796.jpg
written: 21797.jpg
written: 21798.jpg
written: 21799.jpg
written: 21800.jpg
written: 21801.jpg
written: 21802.jpg
written: 21803.jpg
written: 21804.jpg
written: 21805.jpg
written: 21806.jpg
written: 21807.jpg
written: 21808.jpg
written: 21809.jpg
written: 21810.jpg
written: 21811.jpg
written: 21812.jpg
written: 21813.jpg
written: 21814.jpg
written: 21815.jpg
written: 21816.jpg
written: 218

written: 22198.jpg
written: 22199.jpg
written: 22200.jpg
written: 22201.jpg
written: 22202.jpg
written: 22203.jpg
written: 22204.jpg
written: 22205.jpg
written: 22206.jpg
written: 22207.jpg
written: 22208.jpg
written: 22209.jpg
written: 22210.jpg
written: 22211.jpg
written: 22212.jpg
written: 22213.jpg
written: 22214.jpg
written: 22215.jpg
written: 22216.jpg
written: 22217.jpg
written: 22218.jpg
written: 22219.jpg
written: 22220.jpg
written: 22221.jpg
written: 22222.jpg
written: 22223.jpg
written: 22224.jpg
written: 22225.jpg
written: 22226.jpg
written: 22227.jpg
written: 22228.jpg
written: 22229.jpg
written: 22230.jpg
written: 22231.jpg
written: 22232.jpg
written: 22233.jpg
written: 22234.jpg
written: 22235.jpg
written: 22236.jpg
written: 22237.jpg
written: 22238.jpg
written: 22239.jpg
written: 22240.jpg
written: 22241.jpg
written: 22242.jpg
written: 22243.jpg
written: 22244.jpg
written: 22245.jpg
written: 22246.jpg
written: 22247.jpg
written: 22248.jpg
written: 22249.jpg
written: 222

written: 22635.jpg
written: 22636.jpg
written: 22637.jpg
written: 22638.jpg
written: 22639.jpg
written: 22640.jpg
written: 22641.jpg
written: 22642.jpg
written: 22643.jpg
written: 22644.jpg
written: 22645.jpg
written: 22646.jpg
written: 22647.jpg
written: 22648.jpg
written: 22649.jpg
written: 22650.jpg
written: 22651.jpg
written: 22652.jpg
written: 22653.jpg
written: 22654.jpg
written: 22655.jpg
written: 22656.jpg
written: 22657.jpg
written: 22658.jpg
written: 22659.jpg
written: 22660.jpg
written: 22661.jpg
written: 22662.jpg
written: 22663.jpg
written: 22664.jpg
written: 22665.jpg
written: 22666.jpg
written: 22667.jpg
written: 22668.jpg
written: 22669.jpg
written: 22670.jpg
written: 22671.jpg
written: 22672.jpg
written: 22673.jpg
written: 22674.jpg
written: 22675.jpg
written: 22676.jpg
written: 22677.jpg
written: 22678.jpg
written: 22679.jpg
written: 22680.jpg
written: 22681.jpg
written: 22682.jpg
written: 22683.jpg
written: 22684.jpg
written: 22685.jpg
written: 22686.jpg
written: 226

written: 23073.jpg
written: 23074.jpg
written: 23075.jpg
written: 23076.jpg
written: 23077.jpg
written: 23078.jpg
written: 23079.jpg
written: 23080.jpg
written: 23081.jpg
written: 23082.jpg
written: 23083.jpg
written: 23084.jpg
written: 23085.jpg
written: 23086.jpg
written: 23087.jpg
written: 23088.jpg
written: 23089.jpg
written: 23090.jpg
written: 23091.jpg
written: 23092.jpg
written: 23093.jpg
written: 23094.jpg
written: 23095.jpg
written: 23096.jpg
written: 23097.jpg
written: 23098.jpg
written: 23099.jpg
written: 23100.jpg
written: 23101.jpg
written: 23102.jpg
written: 23103.jpg
written: 23104.jpg
written: 23105.jpg
written: 23106.jpg
written: 23107.jpg
written: 23108.jpg
written: 23109.jpg
written: 23110.jpg
written: 23111.jpg
written: 23112.jpg
written: 23113.jpg
written: 23114.jpg
written: 23115.jpg
written: 23116.jpg
written: 23117.jpg
written: 23118.jpg
written: 23119.jpg
written: 23120.jpg
written: 23121.jpg
written: 23122.jpg
written: 23123.jpg
written: 23124.jpg
written: 231

written: 23511.jpg
written: 23512.jpg
written: 23513.jpg
written: 23514.jpg
written: 23515.jpg
written: 23516.jpg
written: 23517.jpg
written: 23518.jpg
written: 23519.jpg
written: 23520.jpg
written: 23521.jpg
written: 23522.jpg
written: 23523.jpg
written: 23524.jpg
written: 23525.jpg
written: 23526.jpg
written: 23527.jpg
written: 23528.jpg
written: 23529.jpg
written: 23530.jpg
written: 23531.jpg
written: 23532.jpg
written: 23533.jpg
written: 23534.jpg
written: 23535.jpg
written: 23536.jpg
written: 23537.jpg
written: 23538.jpg
written: 23539.jpg
written: 23540.jpg
written: 23541.jpg
written: 23542.jpg
written: 23543.jpg
written: 23544.jpg
written: 23545.jpg
written: 23546.jpg
written: 23547.jpg
written: 23548.jpg
written: 23549.jpg
written: 23550.jpg
written: 23551.jpg
written: 23552.jpg
written: 23553.jpg
written: 23554.jpg
written: 23555.jpg
written: 23556.jpg
written: 23557.jpg
written: 23558.jpg
written: 23559.jpg
written: 23560.jpg
written: 23561.jpg
written: 23562.jpg
written: 235

...

written: 25703.jpg
written: 25704.jpg
written: 25705.jpg
written: 25706.jpg
written: 25707.jpg
written: 25708.jpg
written: 25709.jpg
written: 25710.jpg
written: 25711.jpg
written: 25712.jpg
written: 25713.jpg
written: 25714.jpg
written: 25715.jpg
written: 25716.jpg
written: 25717.jpg
written: 25718.jpg
written: 25719.jpg
written: 25720.jpg
written: 25721.jpg
written: 25722.jpg
written: 25723.jpg
written: 25724.jpg
written: 25725.jpg
written: 25726.jpg
written: 25727.jpg
written: 25728.jpg
written: 25729.jpg
written: 25730.jpg
written: 25731.jpg
written: 25732.jpg
written: 25733.jpg
written: 25734.jpg
written: 25735.jpg
written: 25736.jpg
written: 25737.jpg
written: 25738.jpg
written: 25739.jpg
written: 25740.jpg
written: 25741.jpg
written: 25742.jpg
written: 25743.jpg
written: 25744.jpg
written: 25745.jpg
written: 25746.jpg
written: 25747.jpg
written: 25748.jpg
written: 25749.jpg
written: 25750.jpg
written: 25751.jpg
written: 25752.jpg
written: 25753.jpg
written: 25754.jpg
written: 257

written: 26146.jpg
written: 26147.jpg
written: 26148.jpg
written: 26149.jpg
written: 26150.jpg
written: 26151.jpg
written: 26152.jpg
written: 26153.jpg
written: 26154.jpg
written: 26155.jpg
written: 26156.jpg
written: 26157.jpg
written: 26158.jpg
written: 26159.jpg
written: 26160.jpg
written: 26161.jpg
written: 26162.jpg
written: 26163.jpg
written: 26164.jpg
written: 26165.jpg
written: 26166.jpg
written: 26167.jpg
written: 26168.jpg
written: 26169.jpg
written: 26170.jpg
written: 26171.jpg
written: 26172.jpg
written: 26173.jpg
written: 26174.jpg
written: 26175.jpg
written: 26176.jpg
written: 26177.jpg
written: 26178.jpg
written: 26179.jpg
written: 26180.jpg
written: 26181.jpg
written: 26182.jpg
written: 26183.jpg
written: 26184.jpg
written: 26185.jpg
written: 26186.jpg
written: 26187.jpg
written: 26188.jpg
written: 26189.jpg
written: 26190.jpg
written: 26191.jpg
written: 26192.jpg
written: 26193.jpg
written: 26194.jpg
written: 26195.jpg
written: 26196.jpg
written: 26197.jpg
written: 261

written: 26592.jpg
written: 26593.jpg
written: 26594.jpg
written: 26595.jpg
written: 26596.jpg
written: 26597.jpg
written: 26598.jpg
written: 26599.jpg
written: 26600.jpg
written: 26601.jpg
written: 26602.jpg
written: 26603.jpg
written: 26604.jpg
written: 26605.jpg
written: 26606.jpg
written: 26607.jpg
written: 26608.jpg
written: 26609.jpg
written: 26610.jpg
written: 26611.jpg
written: 26612.jpg
written: 26613.jpg
written: 26614.jpg
written: 26615.jpg
written: 26616.jpg
written: 26617.jpg
written: 26618.jpg
written: 26619.jpg
written: 26620.jpg
written: 26621.jpg
written: 26622.jpg
written: 26623.jpg
written: 26624.jpg
written: 26625.jpg
written: 26626.jpg
written: 26627.jpg
written: 26628.jpg
written: 26629.jpg
written: 26630.jpg
written: 26631.jpg
written: 26632.jpg
written: 26633.jpg
written: 26634.jpg
written: 26635.jpg
written: 26636.jpg
written: 26637.jpg
written: 26638.jpg
written: 26639.jpg
written: 26640.jpg
written: 26641.jpg
written: 26642.jpg
written: 26643.jpg
written: 266

written: 27035.jpg
written: 27036.jpg
written: 27037.jpg
written: 27038.jpg
written: 27039.jpg
written: 27040.jpg
written: 27041.jpg
written: 27042.jpg
written: 27043.jpg
written: 27044.jpg
written: 27045.jpg
written: 27046.jpg
written: 27047.jpg
written: 27048.jpg
written: 27049.jpg
written: 27050.jpg
written: 27051.jpg
written: 27052.jpg
written: 27053.jpg
written: 27054.jpg
written: 27055.jpg
written: 27056.jpg
written: 27057.jpg
written: 27058.jpg
written: 27059.jpg
written: 27060.jpg
written: 27061.jpg
written: 27062.jpg
written: 27063.jpg
written: 27064.jpg
written: 27065.jpg
written: 27066.jpg
written: 27067.jpg
written: 27068.jpg
written: 27069.jpg
written: 27070.jpg
written: 27071.jpg
written: 27072.jpg
written: 27073.jpg
written: 27074.jpg
written: 27075.jpg
written: 27076.jpg
written: 27077.jpg
written: 27078.jpg
written: 27079.jpg
written: 27080.jpg
written: 27081.jpg
written: 27082.jpg
written: 27083.jpg
written: 27084.jpg
written: 27085.jpg
written: 27086.jpg
written: 270

written: 27467.jpg
written: 27468.jpg
written: 27469.jpg
written: 27470.jpg
written: 27471.jpg
written: 27472.jpg
written: 27473.jpg
written: 27474.jpg
written: 27475.jpg
written: 27476.jpg
written: 27477.jpg
written: 27478.jpg
written: 27479.jpg
written: 27480.jpg
written: 27481.jpg
written: 27482.jpg
written: 27483.jpg
written: 27484.jpg
written: 27485.jpg
written: 27486.jpg
written: 27487.jpg
written: 27488.jpg
written: 27489.jpg
written: 27490.jpg
written: 27491.jpg
written: 27492.jpg
written: 27493.jpg
written: 27494.jpg
written: 27495.jpg
written: 27496.jpg
written: 27497.jpg
written: 27498.jpg
written: 27499.jpg
written: 27500.jpg
written: 27501.jpg
written: 27502.jpg
written: 27503.jpg
written: 27504.jpg
written: 27505.jpg
written: 27506.jpg
written: 27507.jpg
written: 27508.jpg
written: 27509.jpg
written: 27510.jpg
written: 27511.jpg
written: 27512.jpg
written: 27513.jpg
written: 27514.jpg
written: 27515.jpg
written: 27516.jpg
written: 27517.jpg
written: 27518.jpg
written: 275

In [14]:
    img = image.load_img('test/'+testing.filename[9], target_size=(64,64,3), color_mode='rgb')
    img = image.img_to_array(img)
    img = img/255
    
    data = np.array([img.reshape( (64, 64, 3) )])
        
    get_data = model.predict(data)
    
    get_data[0][0]

0.00051667454